In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from validphys.loader import FallbackLoader as Loader
from validphys.api import API
from collections import defaultdict
from scipy.stats import norm
from validphys.plotutils import kde_plot
from nnpdf_data import legacy_to_new_map


In [ ]:
fit_names = [f"241123-rs-n3lo_3pt_qed-alphas_0{n}" for n in range(114,126)]

In [ ]:
l = Loader()
fits = [l.check_fit(f) for f in fit_names]

# Correlated Replica Method

In [ ]:
as_fits = defaultdict(list)
for f in fits:
    th = f.as_input()["theory"]["theoryid"]
    alpha = API.theory_info_table(theory_db_id = th).loc["alphas"].iloc[0]
    as_fits[alpha].append(f)
as_fits = dict(as_fits)

In [ ]:
indexes = {f: API.fitted_replica_indexes(pdf=f.name) for f in fits}
replica_data = {f: API.replica_data(fit=f.name) for f in fits}

In [ ]:
def measure(replica_data):
    return replica_data.training*3439 + replica_data.validation*1177
    # return replica_data.chi2

In [ ]:
min_values = {}
for alpha, flist in as_fits.items():
    series = []
    for f in flist:
        s = [measure(d) for d in replica_data[f]]
        series.append(pd.Series(s, index=indexes[f]))
    min_values[alpha] = pd.DataFrame(series).min()
data = pd.DataFrame(min_values).dropna()

In [ ]:
# quadratic polynomial
mins = {}
filter_this_row = [] # rows that are filtered
invcov = np.linalg.inv(np.cov(data.T))
for ind, row in data.iterrows():
    p2, p1, p0 = np.polyfit(data.columns, row, 2)
    if not np.isnan(p1): # NaN if not all replicas passed postfit

        # Calculate difference between fitted parabola and datapoint
        y_fit = p2 * data.columns**2 + p1 * data.columns + p0
        diff = (row - y_fit)@invcov@(row - y_fit)
        # if data is non-parabolic, neglect the replica
        # if diff > 10 or p2 < 3500:
        #     filter_this_row.append(ind)
        #     continue

        mins[ind] = -p1 / 2 / p2
mins = pd.Series(mins)
mins_quadratic = pd.Series(mins)

In [ ]:
# Parameters for the bootstrap
n_bootstrap = 1000  # Number of bootstrap samples
ci_percentile = 0.68  # 68% confidence interval

# Function to compute the confidence interval
def compute_ci(series, percentile):
    lower_bound = np.percentile(series, (1 - percentile) / 2 * 100)
    upper_bound = np.percentile(series, (1 + percentile) / 2 * 100)
    return lower_bound, upper_bound

bootstrap_cis = []
for _ in range(n_bootstrap):
    # Resample with replacement
    sample = np.random.choice(mins, size=len(mins), replace=True)
    # Compute confidence interval for the sample
    ci = compute_ci(sample, ci_percentile)
    bootstrap_cis.append(ci)

# Convert to a DataFrame for easier analysis
bootstrap_cis_df = pd.DataFrame(bootstrap_cis, columns=['lower', 'upper'])

# Calculate the bootstrap error (standard deviation of the CIs)
ci_error = bootstrap_cis_df.std()

np.std([j-i for i,j in bootstrap_cis])

In [ ]:
print(mins.describe(percentiles=[0.16,0.84]))
print("")
print(f"cv±std = {mins.mean():.4f} ± {mins.std():.4f} ")
print(f"1std interval:  {mins.mean()-mins.std():.5f} to {mins.mean()+mins.std():.5f} ")
print(f"68% c.i:        {mins.describe(percentiles=[0.16,0.84])[4]:.5f} to {mins.describe(percentiles=[0.16,0.84])[6]:.5f} ")
print(f"68% c.i:        {(mins.describe(percentiles=[0.16,0.84])[4] + mins.describe(percentiles=[0.16,0.84])[6])/2:.4f} ± {(mins.describe(percentiles=[0.16,0.84])[6] - mins.describe(percentiles=[0.16,0.84])[4])/2:.4f} ")

In [ ]:
fig, ax = plt.subplots()
kde_plot(mins,ax=ax)
central = (mins.describe(percentiles=[0.16,0.84])[6] + mins.describe(percentiles=[0.16,0.84])[4])/2
unc = (mins.describe(percentiles=[0.16,0.84])[6] - mins.describe(percentiles=[0.16,0.84])[4])/2
ax.set_title(f"68% c.i: {central:.4f}  ± {unc:.4f}")
# ax.set_xlim(0.118,0.13)
ax.set_xlabel(r"$\alpha_s(M_Z)$")

In [ ]:
plt.hist(mins,bins=data.columns-0.0005,edgecolor='black',density=True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
# p = np.exp(-((x-mins.mean())/mins.std())**2/2)*mins.size/np.sqrt(2*np.pi)
p = norm.pdf(x, mins.mean(), mins.std())
plt.plot(x,p,'k',label=f"{mins.mean():.5f} +/- {mins.std():.5f}")
plt.yticks([])
plt.legend()

In [ ]:
plt.plot(data.columns, data.T/4616, color='blue', lw=0.2)
plt.ylabel(r"$\chi^2$")
plt.xlabel(r"$\alpha_s(M_Z)$")

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

norm = Normalize(vmin=mins.min(), vmax=mins.max())

# Choose the yellow-to-green colormap
colormap = cm.YlGn

# Plot each row of the transposed DataFrame and color them based on 'mins' values
for i, row in enumerate(np.array(data)):
    plt.plot(data.columns, row, color=colormap(norm(mins.iloc[i])))

# Add a colorbar to indicate the mapping from 'mins' values to the color gradient
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
sm.set_array([])
plt.colorbar(sm, label="Mins Values")

In [ ]:
for fitn in fit_names:
    print(API.fit(fit=fitn).as_input()["theory"]["theoryid"])

# Experimental/naive method

In [ ]:
naive_dict = dict(
    fit=fit_names[0],
    dataset_inputs={"from_": "fit"},
    pdf={"from_": "fit"},
    use_cuts="fromfit",
    theory={"from_": "fit"},
    theoryid={"from_": "theory"},
)

# Experimental covariance matrix
# C = API.groups_covmat(
#     use_t0 = False,
#     **naive_dict
# )



# t0 covariance matrix (the correct one, see bottom of page 15 of https://arxiv.org/pdf/1802.03398)
C = API.groups_covmat(
    fit=fit_names[0],
    use_t0 = True,
    use_cuts="fromfit",
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    dataset_inputs={"from_": "fit"},
    theoryid=API.fit(fit=fit_names[0]).as_input()["theory"]["t0theoryid"],
)

# the datapoint is already uniquely defined by the dataset and datapoint, we dont need the process
C = C.droplevel(0, axis=0).droplevel(0, axis=1)


In [ ]:
try:
    stored_covmat = pd.read_csv(
        fits[0].path / "tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
        index_col=[0, 1, 2],
        header=[0, 1, 2],
        sep="\t|,",
        engine="python",
    ).fillna(0)
    tmp = stored_covmat.droplevel(0, axis=0).droplevel(0, axis=1) # drop process level
    new_names = {d[0]: legacy_to_new_map(d[0])[0] for d in tmp.index}
    tmp.rename(columns=new_names, index=new_names, level=0, inplace=True) # rename datasets using the legacy to new map
    tmp_index = pd.MultiIndex.from_tuples(
        [(bb, np.int64(cc)) for bb, cc in tmp.index],
        names=["dataset", "id"],
    ) # make sure the index is an int, just as it is in C
    tmp = pd.DataFrame(
        tmp.values, index=tmp_index, columns=tmp_index
    )
    stored_covmat = tmp.reindex(C.index).T.reindex(C.index)
    if stored_covmat.isnull().values.any():
        print("some values are NaN, meaning that not all indices in C.index exist in tmp")
    invcov = np.linalg.inv(C+stored_covmat)
except:
    invcov = np.linalg.inv(C)

In [ ]:
chi2_values = []
alphas_values = []
for fitname in fit_names:
    naive_dict["fit"] = fitname
    central_preds_and_data = API.group_result_central_table_no_table(**naive_dict)

    theory_db_id = API.fit(fit=fitname).as_input()["theory"]["theoryid"]
    alphas_values.append(API.theory_info_table(theory_db_id = theory_db_id).loc["alphas"].iloc[0])

    # compute chi2
    diff = central_preds_and_data.theory_central - central_preds_and_data.data_central
    chi2_values.append(diff @ invcov @ diff / diff.size)


In [ ]:
a, b, c = np.polyfit(alphas_values, chi2_values, 2)

central = -b / 2 / a
ndata = C.shape[0]
unc = np.sqrt(1/a/ndata)

plt.scatter(alphas_values, chi2_values, color="blue" )
xgrid = np.linspace(min(alphas_values),max(alphas_values))
plt.plot(xgrid, [a*x*x + b*x + c for x in xgrid], color="black", linestyle="--")
plt.title(rf"$\alpha_s$={central:.4f}$\pm${unc:.4f}")
print(f"{central:.4f} ± {unc:.4f}")